In [1]:
import sqlite3
import pandas as pd

In [2]:
bank_table = pd.read_csv('bank.csv')
bank_table.head(3)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998


In [3]:
# Creating a SQLite database connection and writing the DataFrame to a
# SQLite database table
conn = sqlite3.connect('bank.db')
db = conn.cursor()
bank_table.to_sql('bank_table', conn, if_exists='replace', index=False)

10127

Some SQL queries to get to explore the data

In [8]:
# Function to check the size of the dataset
def size_dataset(db):
    query =  "SELECT COUNT(*) FROM bank_table"
    db.execute(query)
    row = db.fetchone()
    return row

print(f'The dataset contains {size_dataset(db)[0]} rows.')

# Function to check the number of unique clients

def unique_clients(db):
    query = "SELECT COUNT(DISTINCT(CLIENTNUM)) FROM bank_table"
    db.execute(query)
    rows = db.fetchall()
    return rows
print(f'The dataset contains {unique_clients(db)[0][0]} unique clients.')

# In both cases, I get 10,127 as a result, which means that effectively the dataset contains the information of unique customers.

The dataset contains 10127 rows.
The dataset contains 10127 unique clients.


In [40]:
# Some basic queries to check the customer profiles/demographics

def gender(db):
    query = "SELECT DISTINCT(Gender), COUNT(*) FROM bank_table GROUP BY Gender"
    db.execute(query)
    rows = db.fetchall()
    return rows

print(gender(db))
print(f'{gender(db)[0][1]} of the clients are females, while {gender(db)[1][1]} are males.')


def avg_age(db):
    query = "SELECT AVG(Customer_Age) FROM bank_table"
    db.execute(query)
    row = db.fetchone()
    return row

print(f'The average age of the clients is {round(avg_age(db)[0])}.')

def avg_age_per_gender(db):
    query = "SELECT DISTINCT(Gender), AVG(Customer_Age) FROM bank_table GROUP BY Gender"
    db.execute(query)
    row = db.fetchall()
    return row

print(avg_age_per_gender(db))

[('F', 5358), ('M', 4769)]
5358 of the clients are females, while 4769 are males.
The average age of the clients is 46.
[('F', 46.45688689809631), ('M', 46.17886349339484)]


In [67]:
def income_category(db):
    query = "SELECT Income_category, count(CLIENTNUM) as customer_count FROM bank_table GROUP BY Income_Category ORDER BY customer_count"
    db.execute(query)
    rows = db.fetchall()
    return rows

#print(income_category(db))
print(f'There are {len(income_category(db))} different types of income categories. They are as follows:\n 1) {income_category(db)[0][0]} with {income_category(db)[0][1]} clients.\n 2) {income_category(db)[1][0]} with {income_category(db)[1][1]} clients.\n 3) {income_category(db)[2][0]} with {income_category(db)[2][1]} clients.\n 4) {income_category(db)[3][0]} with {income_category(db)[3][1]} clients.\n 5) {income_category(db)[4][0]} with {income_category(db)[4][1]} clients.\n 6) {income_category(db)[5][0]} with {income_category(db)[5][1]} clients.')

There are 6 different types of income categories. They are as follows:
 1) $120K + with 727 clients.
 2) Unknown with 1112 clients.
 3) $60K - $80K with 1402 clients.
 4) $80K - $120K with 1535 clients.
 5) $40K - $60K with 1790 clients.
 6) Less than $40K with 3561 clients.


In [109]:
def income_category_gender(db):
    query = """
    SELECT Income_Category,
           SUM(CASE WHEN Gender = 'M' THEN 1 ELSE 0 END) AS M,
           SUM(CASE WHEN Gender = 'F' THEN 1 ELSE 0 END) AS F
    FROM bank_table
    GROUP BY Income_Category
    ORDER BY Income_Category
    """
    db.execute(query)
    rows = db.fetchall()
    return rows

def print_income_category_gender(rows):
    print("{:<20}{:<15}{:<15}".format("Income Category", "Male Count", "Female Count"))
    for row in rows:
        income_category, male_count, female_count = row
        print("{:<20}{:<15}{:<15}".format(income_category, male_count, female_count))

rows = income_category_gender(db)
print_income_category_gender(rows)

print(f'\nThe table above interestingly shows that there are no female clients in the 3 highest income categories. \nThey are only below 60k.')


Income Category     Male Count     Female Count   
$120K +             727            0              
$40K - $60K         776            1014           
$60K - $80K         1402           0              
$80K - $120K        1535           0              
Less than $40K      277            3284           
Unknown             52             1060           

The table above interestingly shows that there are no female clients in the 3 highest income categories. 
They are only below 60k.


In [123]:
def card_category(db):
    query = "SELECT Card_Category, COUNT(*) as avg_credit_lim FROM bank_table GROUP BY Card_Category"
    db.execute(query)
    rows = db.fetchall()
    return rows
results_card_category = card_category(db)

for card in results_card_category:
    print(f'{card[0]} category cards are held by {card[1]} clients.')


Blue category cards are held by 9436 clients.
Gold category cards are held by 116 clients.
Platinum category cards are held by 20 clients.
Silver category cards are held by 555 clients.
